# Libraries

In [ ]:
!nvidia-smi
from google.colab import drive
drive.mount('/content/drive')
%run /content/drive/MyDrive/deepfish/preprocessing_2D.ipynb

In [ ]:
!pip install pytorch-lightning

In [ ]:
from torchsummary import summary
import pytorch_lightning as pl
from IPython.display import clear_output
import nibabel as nib
import csv
import os
import glob 
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
from fastprogress import master_bar, progress_bar
from torchvision import transforms
import torch.optim as optim
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class HistoryLogger(pl.callbacks.Callback):
  def __init__(self, dir = "history.csv"):
    self.dir = dir
  def on_validation_epoch_end(self, trainer, pl_module):
    metrics = trainer.callback_metrics
    if "loss_epoch" in metrics.keys():
      logs = {"epoch": trainer.current_epoch}
      keys = ["loss_epoch", "train_dice_epoch", "train_jac_epoch", "val_loss","val_dice", "val_jac"]
      for key in keys:
        logs[key] = metrics[key].item()
      header = list(logs.keys())
      isFile = os.path.isfile(self.dir)
      with open(self.dir, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        if not isFile:
          writer.writeheader()
        writer.writerow(logs) 
    else:
      pass

def setDropProb(model, prob=0.01):
  for layer in model.modules():
    if isinstance(layer, DropBlock2D):
      layer.drop_prob = prob

clear_output()
############## turn off Debug APIs for Final Training############
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

# Models

## Preparation

In [ ]:
#https://github.com/miguelvr/dropblock/blob/master/dropblock/dropblock.py
class DropBlock2D(nn.Module):
  def __init__(self, drop_prob, block_size):
    super(DropBlock2D, self).__init__()
    self.drop_prob = drop_prob
    self.block_size = block_size

  def forward(self, x):
    # shape: (bsize, channels, height, width)
    assert x.dim() == 4, \
      "Expected input with 4 dimensions (bsize, channels, height, width)"
    if not self.training or self.drop_prob == 0.:
      return x
    else:
      gamma = self.drop_prob / (self.block_size ** 2) #get gamma value
      mask = (torch.rand(x.shape[0], *x.shape[2:], device= x.device) < gamma).float() #sample mask
      block_mask = self._compute_block_mask(mask) #compute block mask
      out = x * block_mask[:, None, :, :] #apply block mask
      out = out * block_mask.numel() / block_mask.sum() #scale output
      return out

  def _compute_block_mask(self, mask):
    block_mask = F.max_pool2d(input=mask[:, None, :, :], kernel_size=(self.block_size, self.block_size), stride=(1, 1), padding=self.block_size // 2)
    if self.block_size % 2 == 0:
      block_mask = block_mask[:, :, :-1, :-1]
    block_mask = 1 - block_mask.squeeze(1)
    return block_mask

class DropBlock3D(DropBlock2D):
  def __init__(self, drop_prob, block_size):
    super(DropBlock3D, self).__init__(drop_prob, block_size)

  def forward(self, x):
    # shape: (bsize, channels, depth, height, width)
    assert x.dim() == 5, \
      "Expected input with 5 dimensions (bsize, channels, depth, height, width)"
    if not self.training or self.drop_prob == 0.:
      return x
    else:
      gamma = self.drop_prob / (self.block_size ** 3) #get gamma value
      mask = (torch.rand(x.shape[0], *x.shape[2:]) < gamma).float() #sample mask
      mask = mask.to(x.device) #place mask on input device
      block_mask = self._compute_block_mask(mask) #compute block mask
      out = x * block_mask[:, None, :, :, :] #apply block mask
      out = out * block_mask.numel() / block_mask.sum() #scale output
      return out

  def _compute_block_mask(self, mask):
    block_mask = F.max_pool3d(input=mask[:, None, :, :, :], kernel_size=(self.block_size, self.block_size, self.block_size), stride=(1, 1, 1), padding=self.block_size // 2)
    if self.block_size % 2 == 0:
      block_mask = block_mask[:, :, :-1, :-1, :-1]
    block_mask = 1 - block_mask.squeeze(1)
    return block_mask

In [ ]:
class conv_block(nn.Sequential):
  def __init__(self, ch_in, ch_out, kernel_size = 3, padding = 1, drop_block=False, block_size = 1, drop_prob = 0):
    super().__init__()
    self.add_module("conv1",nn.Conv2d(ch_in, ch_out, kernel_size, padding = padding,bias=False))
    self.add_module("bn1", nn.BatchNorm2d(ch_out))
    self.add_module("relu1", nn.ReLU(inplace=True))
    self.add_module("conv2",nn.Conv2d(ch_out, ch_out, kernel_size, padding = padding,bias=False))
    if drop_block:
      self.add_module("drop_block", DropBlock2D(block_size = block_size, drop_prob = drop_prob))
    self.add_module("bn2", nn.BatchNorm2d(ch_out))
    self.add_module("relu2", nn.ReLU(inplace=True))

class up_conv(nn.Module):
  def __init__(self,ch_in,ch_out):
    super(up_conv,self).__init__()
    self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(ch_in,ch_out,kernel_size=2,stride=1,padding="same",bias=False,),
		        nn.BatchNorm2d(ch_out),
			      nn.ReLU(inplace=True))

  def forward(self,x):
    x = self.up(x)
    return x

In [ ]:
class Attention_block(nn.Module):
  def __init__(self,F_g,F_l,F_int):
    super(Attention_block,self).__init__()
    self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1,stride=1,padding=0,bias=True),
            nn.BatchNorm2d(F_int))     
    self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1,stride=1,padding=0,bias=True),
            nn.BatchNorm2d(F_int))
    self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1,stride=1,padding=0,bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid())
    self.relu = nn.ReLU(inplace=True)
        
  def forward(self,g,x):
    g1 = self.W_g(g)
    x1 = self.W_x(x)
    psi = self.relu(g1+x1)
    psi = self.psi(psi)
    return x*psi

## U-Net

In [ ]:
class U_Net(nn.Module):
  def __init__(self,img_ch=3,output_ch=2, drop_prob = 0):
    super(U_Net,self).__init__()
        
    self.Maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
    channel = 32
    self.Conv1 = conv_block(ch_in=img_ch,ch_out=channel)
    self.Conv2 = conv_block(ch_in=channel,ch_out=channel*2)
    self.Conv3 = conv_block(ch_in=channel*2,ch_out=channel*4)
    self.Conv4 = conv_block(ch_in=channel*4,ch_out=channel*8, drop_block=True, block_size = 5, drop_prob = drop_prob)
    self.Conv5 = conv_block(ch_in=channel*8,ch_out=channel*16, drop_block=True, block_size = 3, drop_prob = drop_prob)

    self.Up5 = up_conv(ch_in=channel*16,ch_out=channel*8)
    self.Up_conv5 = conv_block(ch_in=channel*16, ch_out=channel*8)

    self.Up4 = up_conv(ch_in=channel*8,ch_out=channel*4)
    self.Up_conv4 = conv_block(ch_in=channel*8, ch_out=channel*4)
        
    self.Up3 = up_conv(ch_in=channel*4,ch_out=channel*2)
    self.Up_conv3 = conv_block(ch_in=channel*4, ch_out=channel*2)
        
    self.Up2 = up_conv(ch_in=channel*2,ch_out=channel)
    self.Up_conv2 = conv_block(ch_in=channel*2, ch_out=channel)

    self.Conv_1x1 = nn.Sequential(
            nn.Conv2d(channel, output_ch,kernel_size=1,stride=1,padding=0), 
            nn.Softmax(dim=1))

  def forward(self,x):
    # encoding path
    x1 = self.Conv1(x)

    x2 = self.Maxpool(x1)
    x2 = self.Conv2(x2)
        
    x3 = self.Maxpool(x2)
    x3 = self.Conv3(x3)

    x4 = self.Maxpool(x3)
    x4 = self.Conv4(x4)

    x5 = self.Maxpool(x4)
    x5 = self.Conv5(x5)

    # decoding + concat path
    d5 = self.Up5(x5)
    d5 = torch.cat((x4,d5),dim=1)
        
    d5 = self.Up_conv5(d5)
        
    d4 = self.Up4(d5)
    d4 = torch.cat((x3,d4),dim=1)
    d4 = self.Up_conv4(d4)

    d3 = self.Up3(d4)
    d3 = torch.cat((x2,d3),dim=1)
    d3 = self.Up_conv3(d3)

    d2 = self.Up2(d3)
    d2 = torch.cat((x1,d2),dim=1)
    d2 = self.Up_conv2(d2)

    d1 = self.Conv_1x1(d2)
    return d1

## Attention U-Net

In [ ]:
class Attention_U_Net(nn.Module):
  def __init__(self,img_ch=3,output_ch=2, drop_prob=0):
    super(AttU_Net,self).__init__()
        
    self.Maxpool = nn.MaxPool2d(kernel_size=2,stride=2)

    self.Conv1 = conv_block(ch_in=img_ch,ch_out=64)
    self.Conv2 = conv_block(ch_in=64,ch_out=128)
    self.Conv3 = conv_block(ch_in=128,ch_out=256)
    self.Conv4 = conv_block(ch_in=256,ch_out=512, drop_block=True, block_size = 5, drop_prob = drop_prob)
    self.Conv5 = conv_block(ch_in=512,ch_out=1024, drop_block=True, block_size = 5, drop_prob = drop_prob)

    self.Up5 = up_conv(ch_in=1024,ch_out=512)
    self.Att5 = Attention_block(F_g=512,F_l=512,F_int=256)
    self.Up_conv5 = conv_block(ch_in=1024, ch_out=512)

    self.Up4 = up_conv(ch_in=512,ch_out=256)
    self.Att4 = Attention_block(F_g=256,F_l=256,F_int=128)
    self.Up_conv4 = conv_block(ch_in=512, ch_out=256)
        
    self.Up3 = up_conv(ch_in=256,ch_out=128)
    self.Att3 = Attention_block(F_g=128,F_l=128,F_int=64)
    self.Up_conv3 = conv_block(ch_in=256, ch_out=128)
        
    self.Up2 = up_conv(ch_in=128,ch_out=64)
    self.Att2 = Attention_block(F_g=64,F_l=64,F_int=32)
    self.Up_conv2 = conv_block(ch_in=128, ch_out=64)

    self.Conv_1x1 = nn.Sequential(
            nn.Conv2d(64, output_ch, kernel_size=1,stride=1,padding=0), 
            nn.Softmax(dim=1))

  def forward(self,x):
    # encoding path
    x1 = self.Conv1(x)

    x2 = self.Maxpool(x1)
    x2 = self.Conv2(x2)
        
    x3 = self.Maxpool(x2)
    x3 = self.Conv3(x3)

    x4 = self.Maxpool(x3)
    x4 = self.Conv4(x4)

    x5 = self.Maxpool(x4)
    x5 = self.Conv5(x5)

    # decoding + concat path
    d5 = self.Up5(x5)
    x4 = self.Att5(d5,x4)
    d5 = torch.cat((x4,d5),dim=1)        
    d5 = self.Up_conv5(d5)
        
    d4 = self.Up4(d5)
    x3 = self.Att4(d4,x3)
    d4 = torch.cat((x3,d4),dim=1)
    d4 = self.Up_conv4(d4)

    d3 = self.Up3(d4)
    x2 = self.Att3(d3,x2)
    d3 = torch.cat((x2,d3),dim=1)
    d3 = self.Up_conv3(d3)

    d2 = self.Up2(d3)
    x1 = self.Att2(d2,x1)
    d2 = torch.cat((x1,d2),dim=1)
    d2 = self.Up_conv2(d2)

    d1 = self.Conv_1x1(d2)
    return d1

In [ ]:
summary(U_Net(), (3,128,128), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             864
       BatchNorm2d-2         [-1, 32, 128, 128]              64
              ReLU-3         [-1, 32, 128, 128]               0
            Conv2d-4         [-1, 32, 128, 128]           9,216
       BatchNorm2d-5         [-1, 32, 128, 128]              64
              ReLU-6         [-1, 32, 128, 128]               0
         MaxPool2d-7           [-1, 32, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          18,432
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11           [-1, 64, 64, 64]          36,864
      BatchNorm2d-12           [-1, 64, 64, 64]             128
             ReLU-13           [-1, 64, 64, 64]               0
        MaxPool2d-14           [-1, 64,

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:443: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)


##preprocessing

In [ ]:
all_train_files = glob.glob("/content/drive/MyDrive/deepfish/Dataset/LargeFish/Fish_Dataset/*/*/*.png")
train_gt_files = glob.glob("/content/drive/MyDrive/deepfish/Dataset/LargeFish/Fish_Dataset/*/* GT/*.png")
train_image_files = list(set(all_train_files) - set(train_gt_files))

train_image_files = np.array(sorted(train_image_files))
train_gt_files = np.array(sorted(train_gt_files))

len(train_image_files), len(train_gt_files)
x_train, x_val, y_train, y_val = train_test_split(train_image_files, train_gt_files, test_size = 0.2, random_state=42)

In [ ]:
# data = np.load("./dataISIC2018/ISIC2018_192_256.npz")
# x,y = data["image"], data["mask"]
# test_size = int((10/100)*x.shape[0])	
# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = test_size, random_state=42)

In [ ]:
train_dataset = DataLoader(FishLoader(x_train, y_train, transform=False), batch_size=16, pin_memory=True,
                        shuffle=True, num_workers=2, 
                        drop_last=True, prefetch_factor = 16)
val_dataset = DataLoader(FishLoader(x_val, y_val, typeData="test"), batch_size=32,
                          num_workers=2, prefetch_factor=16)

In [ ]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = U_Net(drop_prob=0)):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
    #     return loss, dice_score, jaccard_score
    def _step(self, batch):
        image, y_true = batch
        y_val = y_true[:,0]
        y_pred = self.model(image)
        loss = DiceLoss(device=self.device)(y_true, y_pred)
        dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
        return loss, dice_score, jaccard_score

    def training_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"loss": loss, "train_dice": dice_score, "train_jac": jaccard_score}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"val_loss": loss, "val_dice": dice_score, "val_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"test_loss": loss, "test_dice": dice_score, "test_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                         factor = 0.5, patience=15, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_dice"}
        return [optimizer], lr_schedulers
    

In [ ]:
segmentor = Segmentor(U_Net(drop_prob=0))
check_point = pl.callbacks.model_checkpoint.ModelCheckpoint("./sample_data/", filename="ckpt{val_dice:0.4f}",
                                                            monitor="val_dice", mode = "max", save_top_k =1,
                                                            verbose=True, save_weights_only=True,
                                                            auto_insert_metric_name=False,)
progress_bar = pl.callbacks.TQDMProgressBar()
logger = HistoryLogger()
swa = pl.callbacks.StochasticWeightAveraging(swa_epoch_start=25)
PARAMS = {"gpus":1, "benchmark": True, "enable_progress_bar" : False, "logger":False,
        #   "callbacks" : [progress_bar], 
        #    "overfit_batches" :1, 
          "callbacks" : [check_point, progress_bar, logger], 
          "log_every_n_steps" :1, "num_sanity_val_steps":1, "max_epochs":10,
          "precision":16,
          }

trainer = pl.Trainer(**PARAMS)
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weightUnet/current.ckpt") 
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./sample_data/ckpt0.7812.ckpt")


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(segmentor, train_dataset, val_dataset)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params
--------------------------------
0 | model | U_Net | 7.8 M 
--------------------------------
7.8 M     Trainable params
0         Non-trainable params
7.8 M     Total params
15.527    Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/sample_data exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch 7, global step 3599: val_dice was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 8, global step 4049: val_dice was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 9, global step 4499: val_dice reached 0.96269 (best 0.96269), saving model to "/content/sample_data/ckpt0.9627.ckpt" as top 1
